In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - ZipNeRF (Pytorch) Notebook

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_zipnerf.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_zipnerf.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看 GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_zipnerf.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>（建议使用 Python-3 CPU 笔记本）
  </td>
</table>

注意：此笔记本已在以下环境中测试过：

* Python版本= 3.9

## 概述

本笔记本展示了一种[PyTorch实现](https://github.com/SuLvXiangXin/zipnerf-pytorch)的[Zip-NeRF：抗锯齿网格化神经辐射场](https://jonbarron.info/zipnerf/)，用于更有效地渲染神经辐射场（NeRFs）。它主要旨在解决传统NeRF技术的一些局限，传统NeRF技术虽然能够从2D图像创建详细的3D模型，但计算密集且速度较慢。

### 目标

在本教程中，您将学习如何：

- 使用[COLMAP](https://colmap.github.io/)执行运动结构（SfM）来估计场景的三维结构，从一系列二维图像中。
- 使用[Vertex AI custom jobs](https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-custom-job-sample)来校准、训练和渲染NERF场景。
- 使用一系列关键帧照片沿着自定义相机路径渲染视频。

本教程使用以下谷歌云ML服务和资源：

- Vertex AI 训练
- Vertex AI 自定义作业

### 成本

本教程使用 Google Cloud 的计费组件:

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 设置

### 安装

安装以下必需的包以执行此笔记本。

In [ ]:
! pip install --upgrade pip
! pip install google-cloud-aiplatform==1.38.1
! pip install google-cloud-storage==2.14.0
! pip install wget==3.2

### 在开始之前

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

只适用于Colab
如果您使用的是Workbench，则请运行以下命令并跳过此部分。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置谷歌云项目

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得$300的免费信用用于计算和存储成本。

2. [确保为您的项目启用了计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个云存储存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

5. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

### 认证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行认证。请按以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行操作，因为您已经通过认证。

本地JupyterLab实例，请取消注释并运行：

In [ ]:
# ! gcloud auth login

###设置您的项目参数

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

您还可以更改由Vertex AI使用的`REGION`变量。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

创建一个存储桶来存储诸如数据集等中间工件。推荐模式：`gs://cloudnerf-{PROJECT_ID}-unique`

In [ ]:
import os

PROJECT_ID = ""  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

# Enter the name of the bucket without gs://
BUCKET_NAME = ""  # @param {type:"string"}


# Set the project id.
! gcloud config set project {PROJECT_ID}

# Create the bucket if it doesn't already exist.
BUCKET_URI = os.path.join("gs://", BUCKET_NAME)
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Vertex AI SDK 用于 Python

为您的项目初始化 Vertex AI SDK 用于 Python。

In [ ]:
import os

from google.cloud import aiplatform

staging_bucket = os.path.join(BUCKET_URI, "zipnerf_staging")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=staging_bucket)

定义常数

In [ ]:
# The pre-built calibration docker image.
CALIBRATION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-cloudnerf-calibrate:latest"
# The pre-built training docker image.
TRAINING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-cloudnerf-train:latest"
# The pre-built rendering docker image.
RENDERING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-cloudnerf-render:latest"

### 定义常见函数

本节为以下内容定义函数：

- 自定义作业命名

In [ ]:
import subprocess
from datetime import datetime
from typing import Any, List

IMAGE_EXTENSIONS = (".png", ".jpg", ".jpeg", ".gif", ".bmp")
GCS_API_ENDPOINT = "https://storage.cloud.google.com/"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def get_mp4_video_link(mp4_rendering_path: str) -> str:
    # Define the gsutil command.
    command = f"gsutil ls {mp4_rendering_path}"

    # Run the command and capture the output.
    try:
        result = subprocess.check_output(command, shell=True, text=True)
        # Split the result by newlines to get a list of files.
        file_list = result.strip().split("\n")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        file_list = []
    mp4_video_link = file_list[0].replace("gs://", GCS_API_ENDPOINT)
    return mp4_video_link


def write_keyframe_list_to_gcs(
    bucket_path: str, output_gcs_file: str, max_files: int = 10
) -> List[Any]:
    # Get the list of files in the GCS bucket.
    cmd = f"gsutil ls {bucket_path}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("Error listing GCS bucket:", result.stderr)
        return []

    # Filter for image files and extract file names.
    files = result.stdout.splitlines()
    image_files = [
        os.path.basename(f) for f in files if f.lower().endswith(IMAGE_EXTENSIONS)
    ]

    output_file = "out.txt"
    with open(output_file, "w") as file:
        for name in image_files[:max_files]:
            file.write(name + "\n")

    cmd = f"gsutil cp {output_file} {output_gcs_file}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("Error listing GCS bucket:", result.stderr)
        return []

## 准备数据集
有必要准备数据集并将其存储在云存储中。以下示例说明了[mipnerf360](https://jonbarron.info/mipnerf360/)数据集中自行车场景的过程。为了方便起见，mipnerf360中的每个场景都提供了作为单独数据集的独特下载链接。Mip-NeRF 360数据集包含以下7个场景：

- [`自行车`](http://storage.googleapis.com/gresearch/refraw360/bicycle.zip)
- [`盆栽`](http://storage.googleapis.com/gresearch/refraw360/bonsai.zip)
- [`柜台`](http://storage.googleapis.com/gresearch/refraw360/counter.zip)
- [`花园`](http://storage.googleapis.com/gresearch/refraw360/garden.zip)
- [`厨房`](http://storage.googleapis.com/gresearch/refraw360/kitchen.zip)
- [`房间`](http://storage.googleapis.com/gresearch/refraw360/room.zip)
- [`树桩`](http://storage.googleapis.com/gresearch/refraw360/stump.zip)

每个场景都经过COLMAP信息预处理，所以下面部分中的校准步骤是可选的。

In [ ]:
local_mipnerf_data_directory = "./mipnerf360_dataset"  # @param {type:"string"}
MIPNERF_DATA_GCS_PATH = os.path.join(BUCKET_URI, "mipnerf360_dataset")

In [ ]:
# Download the bicycle scene data to a local directory.
! rm -rf $local_mipnerf_data_directory
! mkdir -p $local_mipnerf_data_directory
! wget -P $local_mipnerf_data_directory http://storage.googleapis.com/gresearch/refraw360/bicycle.zip

In [ ]:
# Unzip the mipnerf360 dataset.
! unzip $local_mipnerf_data_directory/bicycle.zip -d $local_mipnerf_data_directory

In [ ]:
# Move mipnerf360 data from local directory to Cloud Storage.
# This step takes a few minutes to finish.
! gsutil -m cp -R $local_mipnerf_data_directory/* $MIPNERF_DATA_GCS_PATH

NERF 管线

### 相机姿态估计
如上所述，Mip-NeRF 360数据集中的所有场景都已经使用colmap信息进行了预处理。然而，为了展示如何在你自己的数据上完整运行管道，我们将使用`bicycle`场景来估计相机姿态。

In [ ]:
# Folder containing all the images of the bicycle scene.
INPUT_IMAGES_FOLDER = f"{MIPNERF_DATA_GCS_PATH}/bicycle/images"  # @param {type:"string"}

# Folder for storing experiment outputs for calibration, training and rendering.
OUTPUT_FOLDER = f"{MIPNERF_DATA_GCS_PATH}/exp/bicycle"  # @param {type:"string"}

一旦数据和实验路径已配置好，运行以下自定义作业。

需要以下参数：

* `use_gpu`：是否使用GPU。
* `gcs_dataset_path`：GCS数据集中图像文件夹的路径。
* `gcs_experiment_path`：存储实验输出的GCS路径。
* `camera`：使用的摄像头类型。透视是 `OPENCV`，鱼眼是 `OPENCV_FISHEYE`。

自定义作业将在`gcs_dataset_path`文件夹中的图像上运行，并将colmap输出存储在`gcs_experiment_path/data`文件夹中。

对于当前数据集中的场景，这一步大约需要30分钟。

In [ ]:
# This job will run colmap camera pose estimation.
data_calibration_job_name = get_job_name_with_datetime("colmap")

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": CALIBRATION_DOCKER_URI,
            "args": [
                "-use_gpu",
                "1",
                "-gcs_dataset_path",
                INPUT_IMAGES_FOLDER,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-camera",
                "OPENCV",
            ],
        },
    }
]

data_calibration_custom_job = aiplatform.CustomJob(
    display_name=data_calibration_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

data_calibration_custom_job.run()

### 训练ZipNeRF模型
一旦Colmap姿态校准完成，我们就可以开始训练。

需要以下参数：

* `gcs_experiment_path`：用于加载处理过的数据集和存储实验结果的GCS路径。
* `gin_config_file`：ZipNeRF网络的配置文件。当前选项包括：
  * configs/360.gin：用于360重建的配置。
  * configs/360_glo.gin：用于带[生成潜在优化]的360重建的配置。
* `factor`：在预处理步骤中对图像进行下采样的因子，影响训练像素地面真实图像和渲染图像的分辨率或细节级别。室内场景建议使用2倍因子，室外场景建议使用4倍因子。**训练中使用的因子必须与渲染中使用的相同。**

自定义作业将在`gcs_experiment_path/data` Colmap数据集中的图像上运行，并将结果输出到`gcs_experiment_path/checkpoints`文件夹中的检查点中。

根据配置不同，此步骤可能需要多达3小时。

In [ ]:
# This job will run zipnerf training.

# This is the nerf training job name. You will use it to load the checkpoints
# in the rendering job for the current run.
nerf_training_job_name = get_job_name_with_datetime("nerf_training")

GIN_CONFIG_FILE = "configs/360.gin"  # @param {type:"string"}
FACTOR = "4"  # @param {type:"string"}

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": TRAINING_DOCKER_URI,
            "args": [
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-gin_config_file",
                GIN_CONFIG_FILE,
                "-factor",
                FACTOR,
            ],
        },
    }
]

nerf_training_custom_job = aiplatform.CustomJob(
    display_name=nerf_training_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_training_custom_job.run(enable_web_access=True)

渲染ZipNeRF模型（360）
完成Colmap姿态校准后，我们可以开始训练。

需要以下参数：

* `gcs_experiment_path`：用于加载处理后数据集和存储实验输出的GCS路径。
* `gin_config_file`：ZipNeRF网络的配置文件。当前选项包括：
  * configs/360.gin：用于360重建的配置。
  * configs/360_glo.gin：用于带有[生成潜在优化](https://www.researchgate.net/publication/318527851_Optimizing_the_Latent_Space_of_Generative_Networks)的360重建的配置。
* `render_video_fps`：渲染视频的帧率。
* `render_path_frames`：需要渲染的路径帧数。
* `factor`：在预处理阶段对降采样图像的影响分辨率或详细级别的因子，影响训练像素地面真值和渲染图像的分辨率。室内场景建议使用2倍因子，室外场景建议使用4倍因子。**训练中使用的因子必须与渲染中使用的相同。**

定制作业将运行在`gcs_experiment_path/data` Colmap数据集中的图像上，并在`gcs_experiment_path/checkpoints`文件夹中输出检查点。

In [ ]:
# This job will run zipnerf rendering.
nerf_rendering_job_name = get_job_name_with_datetime("nerf_rendering")

RENDER_PATH_FRAMES = "120"  # @param {type:"string"}
RENDER_VIDEO_FPS = "30"  # @param {type:"string"}

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": RENDERING_DOCKER_URI,
            "args": [
                "-rendering_job_name",
                nerf_rendering_job_name,
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-gin_config_file",
                GIN_CONFIG_FILE,
                "-render_video_fps",
                RENDER_VIDEO_FPS,
                "-render_path_frames",
                RENDER_PATH_FRAMES,
                "-factor",
                FACTOR,
            ],
        },
    }
]

nerf_rendering_custom_job = aiplatform.CustomJob(
    display_name=nerf_rendering_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_rendering_custom_job.run(enable_web_access=True)

显示来自GCS的渲染视频

In [ ]:
from IPython.display import Video

MP4_RENDERING_PATH = f"{OUTPUT_FOLDER}/render/{nerf_rendering_job_name}/*color.mp4"
mp4_video_link = get_mp4_video_link(MP4_RENDERING_PATH)
Video(mp4_video_link)

渲染ZipNeRF模型（自定义摄像机轨迹）

#### 为渲染自定义相机轨迹创建关键帧文件列表。

要在神经辐射场（NeRF）模型中使用与训练时相同数据集的图片来创建自定义相机轨迹，您可以生成一个关键帧文件列表，其中每个关键帧对应于存储在谷歌云存储（GCS）存储桶中的图像文件的名称。本节将指导您创建此关键帧文件列表。

#### 步骤1：识别关键帧图像
首先，识别您数据集中想要用作关键帧的图像。这些图像应该理想地代表您希望相机轨迹包含的重要视图或角度。

#### 步骤2：创建图像文件名称列表
访问您的GCS存储桶：导航到存储数据集的GCS存储桶。

选择图像文件：选择您希望用作关键帧的特定图像文件。请记住，这些文件应该是用于训练NeRF模型的文件，因为它们已经定义了相应的相机参数。

编译文件名：创建这些选定图像的文件名列表（而不是路径）。确保每个文件名单独在一行上。例如：

In [ ]:
# This job will run zipnerf rendering.
nerf_custom_rendering_job_name = get_job_name_with_datetime("nerf_custom_rendering")

In [ ]:
# Example usage.
KEYFRAME_IMAGE_FILELIST = (
    f"{OUTPUT_FOLDER}/keyframe_list_{nerf_custom_rendering_job_name}.txt"
)
max_files = 30  # Set this to the number of files you want
write_keyframe_list_to_gcs(
    INPUT_IMAGES_FOLDER, KEYFRAME_IMAGE_FILELIST, max_files=max_files
)

#### 运行渲染

一旦训练完成，我们可以进行渲染。

需要以下参数：

* `gcs_experiment_path`：用于加载处理过的数据集和存储实验输出的GCS路径。
* `gin_config_file`：ZipNeRF网络的配置文件。目前的选项有：
  * configs/360.gin：用于360重建的配置。
  * configs/360_glo.gin：使用[生成潜在优化](https://www.researchgate.net/publication/318527851_Optimizing_the_Latent_Space_of_Generative_Networks)的360重建配置。
* `render_video_fps`：渲染视频的帧率。
* `factor`：在预处理步骤中影响训练像素地面真实度和渲染图像分辨率或细节级别的下采样图像的因子。室内场景建议使用2倍因子，室外场景建议使用4倍因子。**训练中使用的因子必须与渲染一致。**
* `keyframe_image_list`：用于渲染自定义相机路径的每行一个图像文件名的列表。

使用关键帧，生成插值路径。该路径代表连接指定关键帧相机姿势的平滑轮廓样条。该过程利用一个预设为默认值30的配置变量`render_spline_n_interp`。因此，最终的插值路径包括`render_spline_n_interp` * (n - 1)个姿势。在讨论的具体场景中，config.render_spline_n_interp配置为30。**如果输入30个关键帧，计算结果为30 * 29，总计870个姿势**。

In [ ]:
# This job will run zipnerf rendering.
# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": RENDERING_DOCKER_URI,
            "args": [
                "-rendering_job_name",
                nerf_custom_rendering_job_name,
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-gin_config_file",
                GIN_CONFIG_FILE,
                "-render_video_fps",
                RENDER_VIDEO_FPS,
                "-factor",
                FACTOR,
                "-gcs_keyframes_file",
                KEYFRAME_IMAGE_FILELIST,
                "-render_path_frames",
                RENDER_PATH_FRAMES,
            ],
        },
    }
]

nerf_custom_rendering_custom_job = aiplatform.CustomJob(
    display_name=nerf_custom_rendering_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_custom_rendering_custom_job.run(enable_web_access=True)

#### 从GCS显示渲染视频

In [ ]:
from IPython.display import Video

MP4_RENDERING_PATH = (
    f"{OUTPUT_FOLDER}/render/{nerf_custom_rendering_job_name}/*color.mp4"
)
mp4_video_link = get_mp4_video_link(MP4_RENDERING_PATH)
Video(mp4_video_link)

## 清理

In [ ]:
# Delete pose estimation, training and rendering custom jobs.
if data_calibration_custom_job.list(
    filter=f'display_name="{data_calibration_job_name}"'
):
    data_calibration_custom_job.delete()
if nerf_training_custom_job.list(filter=f'display_name="{nerf_training_job_name}"'):
    nerf_training_custom_job.delete()
if nerf_rendering_custom_job.list(filter=f'display_name="{nerf_rendering_job_name}"'):
    nerf_rendering_custom_job.delete()
if nerf_custom_rendering_custom_job.list(
    filter=f'display_name="{nerf_custom_rendering_job_name}"'
):
    nerf_custom_rendering_custom_job.delete()